In [1]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingClassifier
import itertools
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from mlxtend.classifier import StackingCVClassifier



warnings.filterwarnings("ignore")
randomseed = 7

Using TensorFlow backend.


In [2]:
x_original = pd.read_csv("../../../XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original=x_original [['Unnamed: 0','L100800','L104600','L103000','S000300','L101700','L100700','FIELD_33',
                       'FIELD_38','FIELD_40','FIELD_31','SEX','AGE',#'CLASS',
                       'FIELD_16','FIELD_23','FIELD_15','FIELD_22','FIELD_17','FIELD_24'
                       ]]

In [3]:
y_original = pd.read_csv("../../../TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original=y_original[['Unnamed: 0','CLASS' ]]

In [4]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)

In [5]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia


print(data.shape)

(169024, 20)


In [6]:
data=data[['L100800','L104600', 'L103000', 'S000300', 'L101700', 'L100700',
       'FIELD_33', 'FIELD_38', 'FIELD_40', 'FIELD_31', 'SEX', 'AGE','CLASS']]
data=data.dropna()
print(data.shape)
data.head()

(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


In [7]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


In [8]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0]
#     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

In [9]:

xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]



In [10]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)


In [11]:
from imblearn.over_sampling import SMOTE,SMOTENC  # doctest: +NORMALIZE_WHITESPACE
randomseed=42

sm = SMOTENC(random_state=randomseed,categorical_features=[6,7,8,9,10],sampling_strategy='minority')
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res


Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


In [31]:
rf_5 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100, 
    max_depth=12,
    min_samples_split=3,
    min_samples_leaf=10,
    max_features="auto",
)

In [76]:
SVC_clf_5=SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [33]:
lr_5=LogisticRegression(random_state=0) 

In [34]:
xgb_model_5 = xgb.XGBClassifier(objective="multi:softmax", random_state=randomseed)

In [92]:
model=make_pipeline(ColumnSelector(cols=(0, 1, 8, 9, 10)), SVC_clf_5 )

In [54]:
from sklearn.model_selection import RandomizedSearchCV

In [64]:

Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'svc__C': Cs, 'svc__gamma' : gammas,'svc__kernel':('linear', 'rbf')}

grid_search = RandomizedSearchCV(model, param_grid, cv=5, verbose=2)
grid_search.fit(xtrain,ytrain)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] svc__kernel=rbf, svc__gamma=0.001, svc__C=10 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... svc__kernel=rbf, svc__gamma=0.001, svc__C=10, total=  38.1s
[CV] svc__kernel=rbf, svc__gamma=0.001, svc__C=10 ....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   38.0s remaining:    0.0s


[CV] ..... svc__kernel=rbf, svc__gamma=0.001, svc__C=10, total=  38.1s
[CV] svc__kernel=rbf, svc__gamma=0.001, svc__C=10 ....................
[CV] ..... svc__kernel=rbf, svc__gamma=0.001, svc__C=10, total=  38.6s
[CV] svc__kernel=rbf, svc__gamma=0.001, svc__C=10 ....................
[CV] ..... svc__kernel=rbf, svc__gamma=0.001, svc__C=10, total=  38.5s
[CV] svc__kernel=rbf, svc__gamma=0.001, svc__C=10 ....................
[CV] ..... svc__kernel=rbf, svc__gamma=0.001, svc__C=10, total=  38.5s
[CV] svc__kernel=linear, svc__gamma=1, svc__C=0.001 ..................
[CV] ... svc__kernel=linear, svc__gamma=1, svc__C=0.001, total=  23.2s
[CV] svc__kernel=linear, svc__gamma=1, svc__C=0.001 ..................
[CV] ... svc__kernel=linear, svc__gamma=1, svc__C=0.001, total=  23.4s
[CV] svc__kernel=linear, svc__gamma=1, svc__C=0.001 ..................
[CV] ... svc__kernel=linear, svc__gamma=1, svc__C=0.001, total=  23.5s
[CV] svc__kernel=linear, svc__gamma=1, svc__C=0.001 ..................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 31.1min finished


SyntaxError: 'return' outside function (<ipython-input-64-538315104edc>, line 8)

In [75]:
print(grid_search.best_params_)
grid_search.best_estimator_

{'svc__kernel': 'rbf', 'svc__gamma': 1, 'svc__C': 1}


Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 8, 9, 10), drop_axis=False)),
                ('svc',
                 SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=1,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [91]:
model=grid_search.best_estimator_
model=SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=1,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False)

In [93]:
model.fit(xtrain,ytrain)

Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 8, 9, 10), drop_axis=False)),
                ('svc',
                 SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [94]:
y_pred = model.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, y_pred))
confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

accuracy 	 0.755


,0,1,2
0,154,45,1
1,48,125,27
2,3,23,174


In [95]:
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76       200
           1       0.65      0.62      0.64       200
           2       0.86      0.87      0.87       200

    accuracy                           0.76       600
   macro avg       0.75      0.76      0.75       600
weighted avg       0.75      0.76      0.75       600



In [96]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,
                         xtrain,ytrain,
                         cv=5,
                         scoring='accuracy',
                        verbose=2)

[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................................. , total=  15.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.2s remaining:    0.0s


[CV] ................................................. , total=  15.5s
[CV]  ................................................................
[CV] ................................................. , total=  15.6s
[CV]  ................................................................
[CV] ................................................. , total=  15.6s
[CV]  ................................................................
[CV] ................................................. , total=  15.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished


In [82]:
scores

array([0.70975586, 0.73088149, 0.73370306, 0.73088149, 0.72407083])

In [73]:
scores

array([0.70761599, 0.73341117, 0.73039502, 0.7288383 , 0.72163845])

In [74]:
np.mean(scores)

0.7243797850170852